<a href="https://colab.research.google.com/github/mmsamiei/just-practice-deep/blob/master/load_with_torchtext_for_que2res.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torchtext
from torchtext.data import Field
import spacy

from spacy.symbols import ORTH
my_tok = spacy.load('en')

def spacy_tok(x):
    return [tok.text for tok in my_tok.tokenizer(x)]

QUERY = Field(lower=True, tokenize=spacy_tok)
RESPONSE = Field(lower=True, tokenize=spacy_tok, is_target=True, init_token='<bos>', eos_token='<eos>')


In [0]:
import torch
if torch.cuda.is_available:
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

In [0]:
my_tok.tokenizer.add_special_case("don't", [{ORTH: "do"}, {ORTH: "n't"}])

In [0]:
from torchtext.data import TabularDataset

train_dataset = TabularDataset(path="./simpleR21.csv", format="CSV",
                               fields=[("query", QUERY),("response", RESPONSE)],
                               csv_reader_params={"delimiter":'\t'})

In [5]:
print(len(train_dataset))
print(train_dataset[0])
print(train_dataset[0].__dict__.keys())
print(train_dataset[10].query)
print(train_dataset[10].response)

64
dict_keys(['query', 'response'])
['what', 'is', 'your', 'favorite', 'film', '?']
['my', 'favorite', 'file', 'is', 'inception']


In [6]:
QUERY.build_vocab(train_dataset, vectors="glove.6B.200d")
RESPONSE.build_vocab(train_dataset)
print(QUERY.vocab.stoi['film'])
print(QUERY.vocab.itos[33])
print(len(QUERY.vocab))

13
religion
66


In [0]:
#QUERY.vocab.extend(RESPONSE.vocab)
#len(QUERY.vocab)
#print(QUERY.vocab.itos[89])

In [0]:
from torchtext.data import BucketIterator

train_iterator = BucketIterator(dataset= train_dataset, batch_size=16
                                ,device=device
                                ,sort_key=lambda x: data.interleave_keys(len(x.query), len(x.response))
                                , repeat = False)

In [9]:
batch = next(iter(train_iterator))
print(batch.__dict__.keys())
print(batch.query.shape)
print(batch.response.shape)
print(batch.input_fields)
print(batch.target_fields)
print(batch.response)

dict_keys(['batch_size', 'dataset', 'fields', 'input_fields', 'target_fields', 'query', 'response'])
torch.Size([6, 16])
torch.Size([12, 16])
['query']
['response']
tensor([[  2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
           2,   2],
        [ 13,   4,   4,   4, 123, 118,   4,   4,  13,  13,  47,   4,  91,   4,
          12,   4],
        [  4,  36,  14,  14,   3,   3,   5,  14,   4,   7,  40,  97,   3,   8,
          20,  23],
        [ 10,  57,  98, 103,   1,   1, 108,  73,  18,  63,   3, 117,   1,  43,
          21,  37],
        [ 19, 100,   6,  78,   1,   1,  74,   6,  19,   9,   1,  48,   1,   3,
          11,  55],
        [  8,   3,   3,  70,   1,   1,   3, 119,   3,  72,   1,   7,   1,   1,
          88,  89],
        [  9,   1,   1,   6,   1,   1,   1,   6,   1,  84,   1,  18,   1,   1,
           3,   3],
        [  3,   1,   1,   3,   1,   1,   1,   3,   1,  75,   1,   9,   1,   1,
           1,   1],
        [  1,   1,   1,   1,   1,   1,   1,

In [10]:
for batch in (iter(train_iterator)):
  print(batch.query.shape)

torch.Size([6, 16])
torch.Size([12, 16])
torch.Size([6, 16])
torch.Size([7, 16])
